# Spaceship Titanic Competition

### Goals: set up, do some data exploration, create a model with no nulls

To Do List - check if model can absorb nulls, if not convert floats to int (can't do that until there are no nulls)

In [1]:
import pandas as pd

final_data = pd.read_csv('test.csv')
train_data = pd.read_csv('train.csv')

In [2]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB


In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [4]:
c = train_data._get_numeric_data()
c.dropna(axis=1, how='any', inplace=True)
# c.drop(['PassengerId', 'Transported'], axis=1, inplace=True)
c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   Transported  8693 non-null   bool 
dtypes: bool(1)
memory usage: 8.6 KB


### Lessons Learned! Every column has nulls and taking them all out is pretty pointless!

Okay, I am going to build a model with just numeric data, inputing nulls

In [5]:
numdata = train_data._get_numeric_data()
numdata.drop(['Transported'], axis=1, inplace=True)

features = numdata.columns.to_list()
numdata

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,39.0,0.0,0.0,0.0,0.0,0.0
1,24.0,109.0,9.0,25.0,549.0,44.0
2,58.0,43.0,3576.0,0.0,6715.0,49.0
3,33.0,0.0,1283.0,371.0,3329.0,193.0
4,16.0,303.0,70.0,151.0,565.0,2.0
...,...,...,...,...,...,...
8688,41.0,0.0,6819.0,0.0,1643.0,74.0
8689,18.0,0.0,0.0,0.0,0.0,0.0
8690,26.0,0.0,0.0,1872.0,1.0,0.0
8691,32.0,0.0,1049.0,0.0,353.0,3235.0


In [6]:
from sklearn.model_selection import train_test_split

X = train_data[features]
y = train_data.Transported

X_train, X_valid, y_train, y_valid = train_test_split(X, y)

slightly modified code from the kaggle tutorial

In [7]:
from sklearn.ensemble import RandomForestClassifier

(I'm just splitting up a cell here so I can see what is taking so long)

https://stackoverflow.com/questions/67361786/how-to-measure-random-forest-classifier-accuracy

In [8]:
from sklearn.impute import SimpleImputer
from sklearn import metrics

In [9]:
i = SimpleImputer()
i_X_train = pd.DataFrame(i.fit_transform(X_train))
i_X_valid = pd.DataFrame(i.transform(X_valid))

i_X_train.columns = X_train.columns
i_X_valid.columns = X_valid.columns

model = RandomForestClassifier(random_state=1, n_estimators=100, criterion='entropy')
model.fit(i_X_train, y_train)
preds = model.predict(i_X_valid)
print("Accuracy:", metrics.accuracy_score(y_valid, preds))

Accuracy: 0.7874885004599816


Refitting my model to all of my training data

In [10]:
# doing what I did to my data to all my training data
i_X = pd.DataFrame(i.transform(X))
i_X.columns = X.columns

model.fit(i_X, y)

RandomForestClassifier(criterion='entropy', random_state=1)

In [11]:
ff = final_data[features]
i_X_final = pd.DataFrame(i.transform(ff))
i_X_final.columns = final_data[features].columns

final_preds = model.predict(i_X_final)

In [12]:
output = pd.DataFrame({'PassengerId': final_data.PassengerId, 'Transported': final_preds})
output.to_csv('submission2.csv', index=False)